***Demo to demonstrate Agent and SelfQuery Retrieval***

Ask Question - what are the recommendations for Preventive care theme?

Input Preparation
1) Synthetic data Input created with multiple products for Preventive Theme without changing the schema structure
2) Read Recommendations library xl to Pandas dataframe

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [5]:
import pandas as pd
mock_df = pd.read_excel("mock_recomm_library_lat.xlsx")
mock_df

,Product/ Reporting Category*,Observation,Recommendation 1,Recommendation 2,Recommendation 3,Why
0,HealthHive Preventive Care Kit,"70% of our 10,000 members have reported diffic...",Use HealthHive to schedule and track preventiv...,NaN,NaN,Regular screenings detect health issues early....
1,CareConnect Access Platform,"75% of our 5,000 members have expressed a desi...",Use CareConnect to find nearby healthcare prov...,NaN,NaN,Convenient access reduces healthcare barriers....
2,MindWave Behavioral Health App,"90% of our 12,000 members have reported experi...",Use MindWave to access guided meditations and ...,NaN,NaN,Mindfulness practices reduce stress and anxiet...
3,ChronicCare Companion Program,"75% of our 9,000 members have reported feeling...",Use ChronicCare to track medication and appoin...,NaN,NaN,Tracking and coaching improve condition manage...
4,SpecialtyScript Medical Specialty RX,"85% of our 11,000 members have reported diffic...",Use SpecialtyScript to find nearby medical spe...,NaN,NaN,Specialized care addresses complex health need...
5,HealthHero Preventive Care Chatbot,"65% of our 9,000 members have reported having ...",Use HealthHero to ask questions and receive pe...,NaN,NaN,Easy access to information empowers informed d...
6,MindfulMind Behavioral Health App,"85% of our 10,500 members have reported experi...",Use MindfulMind to access guided meditations a...,NaN,NaN,Mindfulness practices reduce stress and anxiet...
7,ChronicCarePlus Chronic Conditions Program,"80% of our 9,500 members have reported feeling...",Use ChronicCarePlus to track medication and ap...,NaN,NaN,Tracking and coaching improve condition manage...
8,CareBridge Access to Care Platform,"70% of our 8,500 members have reported difficu...",Use CareBridge to find nearby healthcare provi...,NaN,NaN,Convenient access reduces healthcare barriers....
9,SpecialtyCare Medical Specialty RX,"90% of our 11,500 members have reported diffic...",Use SpecialtyCare to find nearby medical speci...,NaN,NaN,Specialized care addresses complex health need...


Create Pandas Dataframe Agent & ZeroShot Reasoning

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import pandas as pd
from langchain_openai import OpenAI

Prompt = """
        You have to remember the below product & theme relationship. From the user query find the theme by referring below product & theme relationship and search the dataframe to Provide Product/ Reporting Category and recommendations corresponding to the theme. if you found multiple Product/ Reporting Category for the theme please provide recommendations for all.

            1) HealthHive Preventive care Kit Product/ Reporting Category and HealthHero Preventive Care Chatbot  Product/ Reporting Category are related to Preventive care theme.
            2) CareBridge Access to Care Platform  Product/ Reporting Category and Careconnect Access Platform Kit Product/ Reporting Category are related to Access to care theme.
            3) MindWave Behavioral Kit Product/ Reporting Category is related to Behavioral theme.
            4) ChronicCare Program Kit Product/ Reporting Category is related to Chronic theme
            5) Specialityscript Medical Kit Product/ Reporting Category is related to Pharmacy theme.
       
        """

df_agent = create_pandas_dataframe_agent(OpenAI(temperature=0), mock_df,  prefix=Prompt, verbose=True)

In [7]:
output_resp = df_agent.invoke("what are the recommendations for Preventive care theme?")



> Entering new AgentExecutor chain...
Thought: I should find the theme in the dataframe and then retrieve the corresponding recommendations.
Action: python_repl_ast
Action Input: df[df['Product/ Reporting Category*'].str.contains('Preventive care')]Empty DataFrame
Columns: [Product/ Reporting Category*, Observation, Recommendation 1, Recommendation 2, Recommendation 3, Why]
Index: []I should check for any typos or formatting differences in the dataframe.
Action: python_repl_ast
Action Input: df['Product/ Reporting Category*'].unique()['HealthHive Preventive Care Kit' 'CareConnect Access Platform'
 'MindWave Behavioral Health App' 'ChronicCare Companion Program'
 'SpecialtyScript Medical Specialty RX'
 'HealthHero Preventive Care Chatbot' 'MindfulMind Behavioral Health App'
 'ChronicCarePlus Chronic Conditions Program'
 'CareBridge Access to Care Platform' 'SpecialtyCare Medical Specialty RX'] I should try searching for the theme using a different keyword.
Action: python_repl_ast
Acti

In [9]:
print(output_resp)
print(output_resp['input'])
print(output_resp['output'])

{'input': 'what are the recommendations for Preventive care theme?', 'output': 'The recommendations for the Preventive care theme are to use HealthHive to schedule and track preventive care appointments, receive personalized reminders for recommended screenings and vaccinations, access educational resources to understand the importance of preventive care, and use HealthHero to ask questions and receive personalized responses about preventive care.'}
what are the recommendations for Preventive care theme?
The recommendations for the Preventive care theme are to use HealthHive to schedule and track preventive care appointments, receive personalized reminders for recommended screenings and vaccinations, access educational resources to understand the importance of preventive care, and use HealthHero to ask questions and receive personalized responses about preventive care.


Convert to Langchain document to store in Vectore database

In [14]:
from langchain.docstore.document import Document

In [27]:
recomm_lib_output = []
doc =  Document(page_content="{} {}".format(output_resp['input'], output_resp['output']), metadata={"client_name": "mock_client", "year": "2024"})
recomm_lib_output.append(doc)

convert to embeddings and upload to Vectordb

In [28]:
import chromadb

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
new_client = chromadb.EphemeralClient()
vector_stor = Chroma.from_documents(recomm_lib_output, embeddings, client=new_client, collection_name = "openai_collection")

Unstructured PDF Partition as Data Loader

In [29]:
from langchain_community.document_loaders import UnstructuredPDFLoader


In [31]:
loader = UnstructuredPDFLoader("pdf_files/mock_client_slide_deck_lat.pdf")
recomm_deck = loader.load()
print(recomm_deck)

[Document(page_content='Current Themes and Recommendations\n\nPreventative Care\n\nAccess to Care\n\nBehavioral Health\n\nThemes\n\nPreventative\n\nER utilization and urgent care visits are low • Convenience\n\nER utilization and urgent care visits are low • Convenience\n\ncare increased\n\nTelehealth\n\nutilized by 30% of members\n\n\n\n\n\n\n\n30% of members had a behavioral health conditions 35% of behavioral members are dependents 25% of prescriptions dispensed without PCP visit\n\nRecommendations HealthHive\n\nPreventive Care Kit Use HealthHive to schedule and track preventive care appointments.\n\nCareConnect Access Platform\n\nChronic Conditions\n\nObesity and hypertension were top chronic conditions, both are risk factors for Type 2 diabetes ChronicCare Companion Program\n\nObesity and hypertension were top chronic conditions, both are risk factors for Type 2 diabetes ChronicCare Companion Program\n\nPharmacy\n\nSpecialty drugs were a large part of expenses of high cost claims\

Add Metadata to the Documents

In [32]:
recomm_deck[0].metadata['client_name'] = 'mock_client'
recomm_deck[0].metadata['year'] = '2024'

In [33]:
import chromadb

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
new_client = chromadb.EphemeralClient()
vector_stor = Chroma.from_documents(recomm_deck, embeddings, client=new_client, collection_name = "openai_collection")

Describe the metadata for Self Query Retriever

In [37]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name = "source",
        description="Name of the document",
        type="string",
    ),
    AttributeInfo(
        name = "client_name",
        description="Name of the client",
        type="string",
    ),
    AttributeInfo(
        name = "year",
        description="Document created year",
        type="string",
    ),

]
document_content_desc = "mock client recommendations"
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(llm,
                                        vector_stor,
                                        document_content_desc,
                                        metadata_field_info, search_kwargs={"k":1})

Langchain RAG Prompt

In [38]:
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")
print(rag_prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [39]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import PromptTemplate

prompt = ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'],
    template = "You are an assistant for question answering tasks. Use the following pieces of retrieved context to answer the question. if you don't know the answer just say don't know. \nQuestion \nContext: {context} \nAnswer"))])
print(prompt)

input_variables=['context'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question answering tasks. Use the following pieces of retrieved context to answer the question. if you don't know the answer just say don't know. \nQuestion \nContext: {context} \nAnswer"))]


Invoke RAG Chain

In [42]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = ({"context": retriever, "question": RunnablePassthrough()}
            | rag_prompt
             | llm
             | StrOutputParser()
            )
rag_chain.invoke("What are the Preventive care recommendations in 2024 for mock_client?")

'The preventive care recommendations for mock_client in 2024 include using HealthHive to schedule and track preventive care appointments, receiving personalized reminders for recommended screenings and vaccinations, and accessing educational resources to understand the importance of preventive care. They can also use HealthHero to ask questions and receive personalized recommendations for preventive care. Regular screenings and vaccinations are also recommended to prevent illnesses and outbreaks.'